In [1]:
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.5 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
     |████████████████████████████████| 365 kB 59.6 MB/s 
     |████████████████████████████████| 115 kB 65.0 MB/s 
     |████████████████████████████████| 141 kB 53.9 MB/s 
     |████████████████████████████████| 212 kB 62.4 MB/s 
     |████████████████████████████████| 596 kB 62.9 MB/s 
     |████████████████████████████████| 127 kB 63.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████████████████

In [2]:
!pip install transformers
print('Hello')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hello


In [3]:
from transformers import pipeline

import json
import numpy as np
import time

import evaluate

In [4]:
bDistilBert = False
bRunPred = False

In [5]:
if bDistilBert:
    from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
    model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
else:
    from transformers import BertForQuestionAnswering, AutoTokenizer
    modelname = 'deepset/bert-base-cased-squad2'
    model = BertForQuestionAnswering.from_pretrained(modelname)
    tokenizer = AutoTokenizer.from_pretrained(modelname)    

In [6]:
test_file = open('/content/Covid-QA-more-focused.json')
test_data = json.load(test_file)
print(len(test_data['data']))

42


In [7]:
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer) #, batch=32)

In [8]:
if bRunPred==True:
    num_questions = 0
    num_correct_pred = 0
    questions = []
    answers = []
    predictions = []

    for i in test_data['data']:
      t1 = time.time()
      para = i['paragraphs'][0]
      #print('-----------------')
      #print(para.keys())
      print()
      qas = para['qas']
      context = para['context']
      for eachQA in qas:
        num_questions+=1
        print('num_ques ', num_questions)
        #print('eachQA ', eachQA)
        print('Question is: ', eachQA['question'])
        print('Given Answer (GT) is: ', eachQA['answers'][0]['text'])
        questions.append(eachQA['question'])
        answers.append(eachQA['answers'][0]['text'])

        answer = nlp({
            'question': eachQA['question'],
            'context': context})
        print('Predicted answer is: ', answer['answer'])
        predictions.append(answer['answer'])
        #print('answer ', answer)

        if answer['score'] > 0.1:
          if len(answer['answer']) <= len(eachQA['answers'][0]['text']) + 1:
            if len(answer['answer']) == len(eachQA['answers'][0]['text']) + 1:
              answer['answer'] = answer['answer'][:-1]

          if answer['answer'] in eachQA['answers'][0]['text']:
            num_correct_pred+=1
            print('Correct !!!')

      print('Time taken: ', time.time() - t1)

    print('\n\n Accuracy is ', num_correct_pred/num_questions) # This accuracy is manually calculated

In [9]:
if bRunPred==True:
    print(len(questions)) 
    print(len(answers))
    print(len(predictions))
    print(num_correct_pred)

In [10]:
if bRunPred==True:
    with open("answers_bert.json", "w") as f:
        json.dump(answers, f)

    with open("questions_bert.json", "w") as g:
        json.dump(questions, g)

    with open("predictions_bert.json", "w") as h:
        json.dump(predictions, h)    

In [11]:
if bRunPred==False:
    answers = open('/content/answers_bert.json')
    answers = json.load(answers)
    print(len(answers))

    predictions = open('/content/predictions_bert.json')
    predictions = json.load(predictions)
    print(len(predictions))

828
828


In [12]:
metric = evaluate.load("bertscore") #('exact_match')
#metric.compute(references= answers, predictions= predictions)
results = metric.compute(predictions=predictions, references=answers, lang = 'en') #model_type = "distilbert-base-uncased-distilled-squad") #lang='en')
print(results.keys())

dict_keys(['precision', 'recall', 'f1', 'hashcode'])


In [13]:
print(len(results['f1']))
import numpy as np
print(np.mean(results['precision']))
print(np.mean(results['f1']))
print(np.mean(results['recall']))

828
0.9089321989368125
0.8911559339162808
0.8750455154169009


Using lang='en',

Precision: 0.90

F1: 0.88

Recall: 0.86
